In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 6.0MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://p

In [4]:
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium import common
from tqdm import tqdm
from parsing import get_html

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [6]:
base_url = 'https://market.chelpipe.ru/'
base_html = get_html(base_url)

# browser.implicitly_wait(0.5)
main_list = list()
a = '/production/truby-besshovnye/'
page = 1
new_url = (base_url + a).replace('//', '/')
browser.get(new_url)
while True:
    try:
        table = browser.find_element_by_class_name('table__main')
        print(page)
    except:
        break

    table_html = table.get_attribute('outerHTML')
    table_soup = BeautifulSoup(table_html, 'lxml')
    div_row = table_soup.find_all('div', {'class': 'table-row'})
    for dr in div_row:

        page_product = dr.find('a').get('href')

        try:
            browser.get((base_url + page_product).replace('//', '/'))
        except common.exceptions.TimeoutException:
            print('TimeoutException')
            print((base_url + page_product).replace('//', '/'))
            browser.get((base_url + page_product).replace('//', '/'))
        try:
            product_page = browser.find_element_by_class_name('product-card-main__stats').get_attribute('outerHTML')
        except common.exceptions.NoSuchElementException:
            print('NoSuchElementException')
            print(browser.current_url)
            product_page = browser.find_element_by_class_name('product-card-main__stats').get_attribute('outerHTML')
        cart = BeautifulSoup(product_page, 'lxml')
        data = cart.find_all('div', {'class': 'product-card-main__stats-item-value'})
        article = data[-1].text
        type_of_length = data[-2].text.strip()

        row = dr.find_all('div', {'class': 'table-row__item-value'})

        for_tube = dr.find('div', {'class': 'table-row__item-value-secondary'})
        type_tube = for_tube.find_all('div')[-1].text.strip()
        standard = dr.find('div', {'class': 'table-row__item-value-primary'}).text.split(',')
        steel = row[0].text
        diameter = float(row[1].text.split('/')[0].strip())
        wall = float(row[2].text)
        region = row[3].text
        a_r = row[4].find('span').text.split('/')

        try:
            availability = float(a_r[0].strip())
        except ValueError:
            availability = 0

        try:
            if len(a_r) > 1:
                reserve = float(a_r[-1].strip())
            else:
                reserve = 0

        except ValueError:
            raise ValueError()

        try:
            price = float(row[5].text.replace(u'\xa0', u'').replace('i', '').replace('от', '').replace('руб.', '').strip())
        except ValueError:
            price = 0

        for s in standard:
            main_list.append({'type_tube': type_tube, 'standard': s.replace('\n', '').strip(), 'steel': steel,
                              'diameter': diameter, 'wall': wall, 'region': region,
                              'availability': availability, 'reserve': reserve,
                              'price': price, 'article': article, 'type_of_length': type_of_length})

    page += 1
    browser.get(new_url + f'?PAGEN_1={page}')

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [7]:
json_list = json.dumps(main_list,  ensure_ascii=False).replace('},', '},\n')
with open('truby-besshovnye.json', "w",  encoding='utf-8') as file:
    file.write(json_list)
browser.quit()